In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [2]:
import keras
import numpy as np

path = 'C:/Users/mikhail.galkin/Documents/DataProjects/tutorial_keras/py_keras_by_fchollet/Usadba.txt'
text = open(path, encoding='utf8').read().lower()
print('Corpus length:', len(text))

Corpus length: 419962



Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [3]:
# for check
import random
# Select a text seed at random
start_index = random.randint(0, len(text) - 100)
selected_text = text[start_index: start_index + 1000]
print('RANDOMLY SELECTED PIECE OF TEXT: \n"' + selected_text + '"')

RANDOMLY SELECTED PIECE OF TEXT: 
"ктуганова: я могу сказать)
илья николаевич свинцов: жги
елена сергеевна актуганова: легко заменяются на печень трески в банках. а при очень скромном бюджете- вообще на витамины а, е, д и омега 3 отечественного производства. и даже на рыбий жир)
илья николаевич свинцов: ты так никогда миллиардером не станешь)))
елена сергеевна актуганова: останусь скромным мультимиллиардером, так и быть
ренат ирекович актуганов: юля, 687. как родственники? )
илья николаевич свинцов: все ли в добром здравии?
юля сергеевна шутова: умирают потихоньку
юля сергеевна шутова: ой да ты ж моя заечка
виктор саблин: долой
елена сергеевна актуганова: с языка сняла
виктор саблин: омега 3 в любой жирной рыбе, а половина этой дряни в китае - подделки.. если что-то будешь брать, только в государственных аптеках
виктор саблин: около вас есть аптека недалече
юля сергеевна шутова: или в iherb
илья николаевич свинцов: просто я смотрю народ на экскурсии до 20 тысяч на эти таблеточки пускае

In [4]:
# Length of extracted character sequences
maxlen = 100
# We sample a new sequence every `step` characters
step = 3
# This holds our extracted sequences
sentences = []
# This holds the targets (the follow-up characters)
next_chars = []

In [5]:
# for check
range(0, len(text) - maxlen, step)

range(0, 419862, 3)

In [6]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
print(sentences[11:14], next_chars[11:14])


# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars), '\n', chars)

# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)
print('Indices characters:', len(char_indices), '\n', char_indices)

Number of sequences: 139954
['ещё из эмоциональных фоток. селфи палка на айфоне сработала с третьего раза. у меня эмоция, как сказ', ' из эмоциональных фоток. селфи палка на айфоне сработала с третьего раза. у меня эмоция, как сказал ', ' эмоциональных фоток. селфи палка на айфоне сработала с третьего раза. у меня эмоция, как сказал жен'] ['а', 'ж', 'я']
Unique characters: 213 
 ['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '\xa0', '©', '«', '»', '×', 'ä', 'å', 'è', 'ñ', 'õ', 'ö', 'ú', 'û', 'ā', 'ď', 'ę', 'ě', 'į', 'ķ', 'ņ', 'ō', 'ő', 'œ', 'ř', 'š', 'ţ', 'ť', 'ů', 'ż', '́', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь

In [7]:
# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
# for check
print('x.shape:')
print(x.shape)
print('y.shape:')
print(y.shape)

Vectorization...
x.shape:
(139954, 100, 213)
y.shape:
(139954, 213)


In [8]:
# for check
for i, sentence in enumerate(sentences[11:12]):
    print(i,':len(sentence)=', len(sentence),':', sentence)
    for t, char in enumerate(sentence):
        print(t, char,':char_indices=', char_indices[char])
    print(i,':next_chars=', next_chars[11], ':char_indices[next_chars]=', char_indices[next_chars[11]])

0 :len(sentence)= 100 : ещё из эмоциональных фоток. селфи палка на айфоне сработала с третьего раза. у меня эмоция, как сказ
0 е :char_indices= 99
1 щ :char_indices= 119
2 ё :char_indices= 126
3   :char_indices= 1
4 и :char_indices= 102
5 з :char_indices= 101
6   :char_indices= 1
7 э :char_indices= 123
8 м :char_indices= 106
9 о :char_indices= 108
10 ц :char_indices= 116
11 и :char_indices= 102
12 о :char_indices= 108
13 н :char_indices= 107
14 а :char_indices= 94
15 л :char_indices= 105
16 ь :char_indices= 122
17 н :char_indices= 107
18 ы :char_indices= 121
19 х :char_indices= 115
20   :char_indices= 1
21 ф :char_indices= 114
22 о :char_indices= 108
23 т :char_indices= 112
24 о :char_indices= 108
25 к :char_indices= 104
26 . :char_indices= 15
27   :char_indices= 1
28 с :char_indices= 111
29 е :char_indices= 99
30 л :char_indices= 105
31 ф :char_indices= 114
32 и :char_indices= 102
33   :char_indices= 1
34 п :char_indices= 109
35 а :char_indices= 94
36 л :char_indices= 105
37 к :char_i

In [9]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [10]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

W1002 12:03:07.938650 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1002 12:03:07.968231 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1002 12:03:07.972744 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [11]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

W1002 12:03:21.844123 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1002 12:03:21.860167 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               175104    
_________________________________________________________________
dense_1 (Dense)              (None, 213)               27477     
Total params: 202,581
Trainable params: 202,581
Non-trainable params: 0
_________________________________________________________________


## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    #print('preds coming out of the model::', preds)
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    #print('new preds -->', preds)
    probas = np.random.multinomial(1, preds, 1)
    #print('max(probas)', np.argmax(probas))
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [13]:
import random
import sys

for epoch in range(1, 60):
    print('========================================================================================================')
    print('EPOCH:', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- RANDOMLY SELECTED PIECE OF TEXT::\n"' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ TEMPERATURE::', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

EPOCH: 1


W1002 12:26:17.596101 24160 deprecation.py:323] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1002 12:26:18.980239 24160 deprecation_wrapper.py:119] From C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
139954/139954 [==============================] - 293s 2ms/step - loss: 2.0316
--- RANDOMLY SELECTED PIECE OF TEXT::
"ова: ещё никто не строил империи из двух минусов)) может, вообще надо не базы, как у всех, а в неизв"
------ TEMPERATURE:: 0.2
ова: ещё никто не строил империи из двух минусов)) может, вообще надо не базы, как у всех, а в неизвально столько столько столько соблания себита с на по в собли по вот собести в собесть какие столько столько по вольше себения себита соблить в посто обение стотить на по вот по вот на по восто в столько собли столько собли столько от на таком на такое столько просто в столько столько столько столько себена соворить себита собли столько по сторовальное соблим на такой сторовают и которые по водно 
------ TEMPERATURE:: 0.5
о себена соворить себита собли столько по сторовальное соблим на такой сторовают и которые по водно столим обрасия столько сторания субества по вернал стали
елена сергеевна актуганова: ну на телько соблиное со трисаль ка

C:\Users\mikhail.galkin\AppData\Local\Continuum\anaconda3\envs\py_tensorflow\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ч зарипов: я впервёное и черахникоп2
сегодня? согласиа?
харитонов
EPOCH: 13
Epoch 1/1
139954/139954 [==============================] - 300s 2ms/step - loss: 1.3089
--- RANDOMLY SELECTED PIECE OF TEXT::
"ин: вот эту можно использовать
виктор клевакин: будем надеяться))
михаил юрьевич галкин: да. это сов"
------ TEMPERATURE:: 0.2
ин: вот эту можно использовать
виктор клевакин: будем надеяться))
михаил юрьевич галкин: да. это совстрали на подавила с одного в предлага на полетили подограли подомодлись на под в городически в столице подомодльно под подопали подопале, что такой приезжать, то под подомались под реблили на под на под нашились и подомоджий, что то подомодльно все домогутельного в столит в помобили в городе стало с днем спасти на полетили на под подомались и не показались с подомоджию собака с домой сталом в про
------ TEMPERATURE:: 0.5
с днем спасти на полетили на под подомались и не показались с подомоджию собака с домой сталом в пропал в приздалают под воздное)
илья николаеви